## I. Introduction to the Problem and Dataset

The garment industry is a labor-intensive sector heavily dependent on manual processes. This reliance presents several challenges, including maintaining high levels of productivity, ensuring consistent quality, and managing the workforce efficiently. With increasing competition and the need for rapid turnaround times, optimizing these processes becomes crucial for industry stakeholders. As such, tracking, analyzing, and predicting productivity and performance of factory teams is highly desirable. With that in mind, the problem/task that will be targeted in this notebook is a regression problem focusing on predicting productivity.

## II. Description of the dataset

The Garments dataset is a dataset that contains important attributes of the garment manufacturing process and employee productivity. This dataset was manually collected by Abdulla Al Imran and then validated by industry experts. As the data was manually collected, there is a possibility that the accuracy and reliability of the data is affected. However, as the data was validated by industry experts, the impact on accuracy and reliability of the data may be lessened. 

In the dataset, each row represents an instance identified by a combination of three features: date, team, and department. Meanwhile, each column represents a feature. There are a total of 1197 instances in this dataset and 15 features. The features are as follows:

- date: the working day from when the data is from
- quarter: the portion of the month. In the context of this dataset, it serves as 7 day intervals of a month meaning months reaching over 28 days may have a 5th quarter.
- department: the department associated with the rest of the data of the instance
- day: the day of the week
- team: the team number ass
- targeted_productivity: the targeted productivity set by those in charge of each team for the day.
- smv: standard minute value, the amount of time required to complete a task under standard working conditions
- wip: work in progress, the amount of items that are still being worked on as well as unfinished items
- over_time: the amount of overtime spent by each team in total in minutes
- incentive: the amount of financial incentive (specifically BDT) used to motivate workers
- idle_time: the amount of times when production was interrupted during the day
- idle_men: the number of workers who were idle because of production interruption
- actual_productivity: the % of productivity delivered by the workers (0-1 scale)


## III. List of requirements

Set the path for the dataset as well as the random_seed to be used

In [ ]:
# Constants
dataset_path = 'Dataset 2 - Garments Dataset/garments.csv'
random_seed = 69

Libraries used include:

- pandas
- numpy
- matplotlib
- itertools
- seaborn
- sklearn
- joblib

In [ ]:
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import seaborn as sns

# Training and Testing the model
from sklearn.model_selection import train_test_split # For splitting the dataset into training and testing
from sklearn.pipeline import Pipeline # For creating a pipeline
from sklearn.compose import ColumnTransformer # For transforming the columns
from sklearn.preprocessing import StandardScaler # For Standardization
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV # For Hyperparameter Tuning
from sklearn.preprocessing import PolynomialFeatures  # For Polynomial Regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score   # For evaluation
import joblib   # For saving the model
from sklearn.base import BaseEstimator, RegressorMixin # For creating a custom regressor

# Models
from sklearn.linear_model import LinearRegression # For Linear Regression
from sklearn.linear_model import Lasso # For Lasso Regression
from sklearn.linear_model import SGDRegressor # For Stochastic Gradient Descent
from sklearn.neural_network import MLPRegressor # For Neural Network
from sklearn.ensemble import RandomForestRegressor

# Load the dataset from Dataset folder
og_df = pd.read_csv(dataset_path)
df = pd.read_csv(dataset_path)

pd.set_option('display.max_columns', None) # Display all columns

## IV. Data Preprocessing and Cleaning


Check for null values

In [ ]:
# check for NaN values
print(df.isnull().sum())

In [ ]:
# check values in wip
df['wip'].value_counts()

Fill in the null values with the mean of the column. Setting the mean as the fill value instead of 0 is done as the feature is 'WIP', a feature representing the unfinished items at some stage of production in the garment factory. Since it's improbable that the WIP value is zero (as long as production is ongoing), filling null values with the column's mean is a reasonable approach

In [ ]:
# fill in the missing values with the mean of the column
df['wip'].fillna(df['wip'].mean(), inplace=True)
df['wip'].value_counts()

The department column has typos and inconsistencies so it will be cleaned.

In [ ]:
# check department column
print(df['department'].unique())

# change all 'finishing ' to 'finishing'
df['department'] = df['department'].replace('finishing ', 'finishing')
df['department'] = df['department'].replace('sweing', 'sewing')
print(df['department'].unique())

Checking the data type of all the columns to ensure that they are in the correct format.

In [ ]:
#check datatypes for all columns
print(df.dtypes)

We check the values of the features to ensure that there are no anomalies in the data.

In [ ]:
print(df['department'].value_counts())

In [ ]:
print(df['quarter'].value_counts())

In [ ]:
print(df['day'].value_counts())

In [ ]:
print(df['team'].value_counts())

In [ ]:
print(df['targeted_productivity'].value_counts())

In [ ]:
print(df['smv'].value_counts())

In [ ]:
print(df['over_time'].value_counts())

There are instances of no_of_workers having half values. This is not possible in a real-world scenario so the values will be rounded down to the nearest whole number. We round down as a conservative estimation to avoid overestimating the number of workers.

In [ ]:
print(df['no_of_workers'].head())
# round down the no_of_workers
df['no_of_workers'] = np.floor(df['no_of_workers'])
print(df['no_of_workers'].head())

Check if there are any instances where actual productivity exceeds the 0-1 range

In [ ]:
print(df['actual_productivity'].value_counts())
# print actual_productivity where it is < 0 or > 1
print(df[(df['actual_productivity'] < 0) | (df['actual_productivity'] > 1)])

Since actual productivity has values exceeding 1, we round them to 1. We do this since the dataset description states that the range of actual productivity is 0-1. This is done to ensure that the model is trained on the correct range of values.

In [ ]:
# round down the actual_productivity if it is > 1
df['actual_productivity'] = np.where(df['actual_productivity'] > 1, 1, df['actual_productivity'])
print(df[(df['actual_productivity'] < 0) | (df['actual_productivity'] > 1)])

We drop the date column as the data only spans a single season, making it unlikely to provide meaningful variability or insights for analysis.

In [ ]:
#drop date
df = df.drop('date', axis=1)

In [ ]:
# check for duplicates
print(df.duplicated().sum())

In [ ]:
#check if any numerical value is less than 0
print(df[(df.select_dtypes(include=[np.number]) < 0).any(axis=1)])

In [ ]:
# print all datatypes
print(df.dtypes)

We check for large over_time values but do not remove them yet as the data may be valid and the numbers are believable.

In [ ]:
# calculate overtime per worker
df['overtime_per_worker'] = df['over_time'] / df['no_of_workers']
print(df[df['overtime_per_worker'] > 120].shape)
print(df[df['overtime_per_worker'] > 240].shape)
print(df[df['overtime_per_worker'] > 420].shape)
print(df[df['overtime_per_worker'] > 600].shape)

In [ ]:
# print all idle_men with decimal values
print(df[df['idle_men'] % 1 != 0])

In [ ]:
#print all rows with idle time = 0 and idle_men > 0
print(df[(df['idle_time'] == 0) & (df['idle_men'] > 0)])

Since department, day, and quarter are categorical variables, we one-hot encode them. 

In [ ]:
# turn department, day, and quarter into one-hot encoding
df_cleaned = df.copy()
df = pd.get_dummies(df, columns=['department', 'day', 'quarter'])
# print all datatypes
print(df.dtypes)

We check for days where there are no incentives but there is overtime. However, we do not remove these instances as incentives may not be the only reason for overtime.

In [ ]:
#check for days where there are no incentives but have overtime
df[(df['incentive'] == 0) & (df['over_time'] > 0)][['targeted_productivity', 'idle_men', 'idle_time', 'actual_productivity', 'incentive', 'over_time', 'wip', 'smv']]

In [ ]:
#check cleaned dataset
print(df.info())

# V. Exploratory Data Analysis

## V-A. Correlation and Distribution of Numerical Features (Seal)

A histogram and boxplot is used to determine the distribution of the data and outlier detection for all numerical features in the data. As observed below, numerical features in the dataset have significantly different ranges.  Large-scale differences can bias these algorithms toward features with larger magnitudes.

Hence, there is a major Consideration to use Standard Scaler as it centers the data at 0 and scales all features to a standard deviation of 1, ensuring all features contribute equally to the model.


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
def plot_numerics(data):
    numeric_columns = data.columns
    # Plotting histograms and box plots for each numeric column
    for column in numeric_columns:
        _, ax = plt.subplots(1,2, figsize=(16, 5))
        ax=ax.flatten()
        # Histogram
        sns.histplot(data[column], bins=50, kde=True, color='skyblue', ax=ax[0])
        ax[0].set_title(f'Histogram of {column}', fontsize=15,fontweight='bold')
        ax[0].set_xlabel(column, fontsize=12)
        ax[0].set_ylabel('Frequency', fontsize=12)
        # Box plot
        sns.boxplot(x=data[column], color='#FFEE8C', ax=ax[1])
        ax[1].set_title(f'Box plot of {column}', fontsize=15,fontweight='bold')
        ax[1].set_xlabel(column, fontsize=12)
        plt.tight_layout()
        plt.show()

In [ ]:
df.describe()

Here, we will plot the numerical features to see the distribution of the data and to check for outliers.

In [ ]:
num_cols = ['targeted_productivity', 'smv', 'wip', 'over_time', 'incentive', 'idle_time', 'actual_productivity', 'overtime_per_worker']
plot_numerics(df[num_cols])

The distribution of targeted productivity implies that teams are expected to perform at a high level, with majority of the data points falling between 0.7 and 0.8. As these are target values in a production setting, it is expected that they are set at a high level to ensure production efficiency.

On the other hand, actual productivity is more spread out, with a wider range of values possibly indicating that teams are unable to meet the high targets set for them. This could be due to various factors such as lack of incentives, high idle time, or high overtime.

Incentive and idle_time have mostly 0 values. This indicates that incentives are rarely given and production interruptions are infrequent.

Finally, overtime_per_worker shows that teams tend to work overtime, with most values falling between 100 to 200 minutes.

Now, we will look at the correlation between the numerical features to see if there are any strong correlations between the features.

In [ ]:
correlation_matrix = df[num_cols].corr()
plt.figure(figsize=(20, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix", fontsize=16)
plt.show()

Based from the correlation matrix, we can see that most of the features have a weak correlation with each other. The highest correlation is between 'over_time' and 'smv' implying that the workers are more likely to work overtime if the tasks for the day are more time-consuming.

## V-B. Productivity with regards to Teams and Departments (Orrin)

In this section, we want to look at how productivity varies with respect to teams and departments.

In [ ]:
# actual productivity vs department
plt.figure(figsize=(20, 10))
sns.boxplot(x='department', y='actual_productivity', data=df_cleaned)
plt.title("Actual Productivity vs Department", fontsize=16)
plt.show()

We can see that the finishing department has a higher median productivity than the sewing department but it also has higher variability. Furthermore, the sewing department has multiple outliers on the lower end of the productivity scale.

To gain more insights, we look into the productivity of each team in the sewing and finishing departments. 

In [ ]:
# actual productivity vs team by department
plt.figure(figsize=(20, 10))
sns.boxplot(x='team', y='actual_productivity', hue='department', data=df_cleaned)
plt.title("Actual Productivity vs Team by Department", fontsize=16)
plt.show()

The finishing department consistently shows higher median productivity with less variability, while the sewing department has lower productivity and more extreme outliers, indicating greater inconsistency. 

Team 6 of the finishing department and team 5 of the sewing department stand out as underperformers.

Since there is a target productivity set for each team, we can compare the actual productivity with the target productivity to see how well each team is performing relative to the target.

In [ ]:
# productivity diff per team by department
df_cleaned['productivity_difference'] = df_cleaned['targeted_productivity'] - df_cleaned['actual_productivity']
plt.figure(figsize=(20, 10))
sns.boxplot(x='team', y='productivity_difference', hue='department', data=df_cleaned)
plt.title("Productivity Difference vs Team by Department", fontsize=16)
plt.show()

Despite standing out as having less productivity, team 6 of the finishing department and team 5 of the sewing department tend to reach their target productivity. On the other hand, higher productivity teams like team 1 of both departments tend to fall short of their target productivity.

Now we check the distribution of workers in each department to see if there are any imbalances in the number of workers in each department.

In [ ]:
# number of workers per team by department in table format
workers_per_team = df_cleaned.groupby(['team', 'department'])['no_of_workers'].mean().reset_index()
workers_per_team = workers_per_team.pivot(index='team', columns='department', values='no_of_workers')
workers_per_team

Apart from teams 6 and 12, the number of workers in each team is relatively balanced. So we look at other factors that may influence productivity such as incentives.

In [ ]:
#could the issue be related to incentives?
df2 = df_cleaned.copy()
# remove incentive outliers
df2 = df2[df2['incentive'] < 700]

plt.figure(figsize=(20, 10))
sns.boxplot(x='team', y='incentive', hue='department', data=df2)
plt.title("incentive per team by department", fontsize=16)
plt.show()

It seems that the finishing department rarely gets incentives while the sewing department gets incentives more frequently. Though it seems that incentives do not have a significant impact on whether or not a team reaches its target productivity. However, there does seem to be a slight positive correlation between incentives and productivity.

To further dive into why some teams are unable to meet their target productivity, we look at the SMV of each team.

In [ ]:
#perhaps the issue is in the smv
plt.figure(figsize=(20, 10))
sns.boxplot(x='team', y='smv', hue='department', data=df_cleaned)
plt.title("SMV by team by department", fontsize=16)
plt.show()

We can see that the SMV of the sewing department is generally higher than that of the finishing department. This explains the larger workforce in the sewing department as the tasks are more time-consuming. However, SMV does not seem to have a significant impact on productivity based on the graphs.

## V-C. Day of the Week and Quarter of the Month to Actual Productivity (Tean)

### Description

In the Philippines, a company typically pays its workers at the end of the 2nd and 4th week of every month. With this, it can be speculated that the actual productivity of the workers will be affected when in the proximity of those times as they anticipate receiving their paychecks. Additionally, with the chosen representation, it will also be possible to analyze actual productivity as the month goes by.

Relevant fields of the EDA:

Days of the Week:
- 'day_Monday'
- 'day_Tuesday'
- 'day_Wednesday'
- 'day_Thursday'
- 'day_Saturday'
- 'day_Sunday'

Quarters of the Month:
- 'quarter_Quarter1'
- 'quarter_Quarter2'
- 'quarter_Quarter3'
- 'quarter_Quarter4'
- 'quarter_Quarter5'

Actual Productivity:
- 'actual_productivity'

### Data Transformation

Seclude the relevant fields for the EDA.

In [ ]:
day_quarter_productivity_df = df[['day_Monday', 'day_Tuesday', 'day_Wednesday', 'day_Thursday', 'day_Saturday', 'day_Sunday', 'quarter_Quarter1', 'quarter_Quarter2', 'quarter_Quarter3', 'quarter_Quarter4', 'quarter_Quarter5', 'actual_productivity']]

day_quarter_productivity_df

Transform the relevant fields into more easily processable values.

In [ ]:
# Ensure we are working with a copy of the DataFrame
day_quarter_productivity_df = day_quarter_productivity_df.copy()

# Make day_of_week and quarter columns
day_quarter_productivity_df.loc[:, 'day_of_week'] = day_quarter_productivity_df[['day_Monday', 'day_Tuesday', 'day_Wednesday', 'day_Thursday', 'day_Saturday', 'day_Sunday']].idxmax(axis=1).str.split('_').str[1]
day_quarter_productivity_df.loc[:, 'quarter'] = day_quarter_productivity_df[['quarter_Quarter1', 'quarter_Quarter2', 'quarter_Quarter3', 'quarter_Quarter4', 'quarter_Quarter5']].idxmax(axis=1).str.split('_').str[1]

# Drop the one-hot encoded columns
day_quarter_productivity_df = day_quarter_productivity_df.drop(['day_Monday', 'day_Tuesday', 'day_Wednesday', 'day_Thursday', 'day_Saturday', 'day_Sunday', 'quarter_Quarter1', 'quarter_Quarter2', 'quarter_Quarter3', 'quarter_Quarter4', 'quarter_Quarter5'], axis=1)

day_quarter_productivity_df

### Data Representation and Visualization

Pivot Tables specialize in representing grouped data of many dimensions into a neat table format, where the rows and columns are the groupings and the cells are the featured values.
As the EDA features 1 quantitative and 2 qualitative features, a Pivot Table should be the most fitting representation. 
This lets us create a table who's columns are the days of the week, rows are the quarters of the month, and cells are the means of the actual productivities grouped by the columns and rows.

In [ ]:
# Create a pivot table
pivot_table = day_quarter_productivity_df.pivot_table(
    values='actual_productivity', 
    index='quarter', 
    columns='day_of_week', 
    aggfunc='mean'
)

# Sort the days of the week
pivot_table = pivot_table[['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Saturday']]

pivot_table

To better visualize the Pivot Table, a Heat Map can be made to highlight the trends of the data in each grouping.

In [ ]:
# Plot the pivot table
plt.figure(figsize=(8, 6))
plt.imshow(pivot_table, cmap='viridis')
plt.colorbar()
plt.xticks(np.arange(6), pivot_table.columns)
plt.yticks(np.arange(5), pivot_table.index)
plt.xlabel('Day of the Week')
plt.ylabel('Quarter of the Month')
plt.title('Average Productivity by Day of the Week and Quarter of the Month')
plt.show()

### Findings

- Actual productivity of workers decreases as the month goes by, with the later half of the month being darker than the start. 
- The start of Quarter 3 (0.677855) and the end of Quarter 4 (0.654613) are especially dark, suggesting that the workers are less productive after receiving their 1st monthly paycheck and before receiving their 2nd monthly paycheck.
- Quarter 5 is very productive as its colors are way brighter. However, this may be an outlier due to the only Quarter 5 days being 1/29/2015 (0.791633) and 1/31/2025 (0.854926).

## V-D. Overtime in relation to Standard Minute Value and Number of Workers for each Department (Tan Ai)

### Description

Considering standard minute value was the amount of time it'd take to complete 1 work of an order, I was interested in seeing how overtime occurs in both departments based on the manpower they have, I split it into the two separate departments as the values for SMV varied greatly between departments, sensible enough as the finishing departments work such as trimming, button attachment, ironing, etc. are much less work (“Functions of Finishing Department in Garment Industry,” 2015) when compared to the lengthy process of sewing the garments together.

Relevant EDA fields:
- Standard Minute Value (smv)
- Overtime (over_time)
- Number of Workers (no_of_workers)

In [ ]:
df.describe()

In [ ]:
sewing_smv_nw_overtime_df = df[(df["department_sewing"] == True)][['smv', 'no_of_workers', 'over_time']]

finishing_smv_nw_overtime_df = df[(df["department_finishing"] == True)][['smv', 'no_of_workers', 'over_time']]

In [ ]:
sewing_smv_nw_overtime_df.head()


In [ ]:
sewing_smv_nw_overtime_df = sewing_smv_nw_overtime_df.copy()

finishing_smv_nw_overtime_df = finishing_smv_nw_overtime_df.copy()

### Data Representation and Visualization

3D Scatterplots allow the relationship between variables to be observed in 3d space, considering that we have 3 quantitative variables this method of visualization should be fitting.

In [ ]:
x = finishing_smv_nw_overtime_df['smv']
y = finishing_smv_nw_overtime_df['no_of_workers']
z = finishing_smv_nw_overtime_df['over_time']

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(x, y, z, c=z, cmap='viridis')

ax.set_xlabel('Standard Minute Value')
ax.set_ylabel('Number of Workers')
ax.set_zlabel('Overtime')

plt.show()

In [ ]:
x = sewing_smv_nw_overtime_df['smv']
y = sewing_smv_nw_overtime_df['no_of_workers']
z = sewing_smv_nw_overtime_df['over_time']

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(x, y, z, c=z, cmap='viridis')

ax.set_xlabel('Standard Minute Value')
ax.set_ylabel('Number of Workers')
ax.set_zlabel('Overtime')

plt.show()

### Findings

- In both departments there are outliers that have an extreme amount of overtime.
- The number of workers in the finishing department is lower than the number in the sewing department
- Overtime has a tendency to become lower when the smv is low and there are more workers for the sewing department.
- Despite average smv in the finishing department, the number of workers seem to correlate with increasing overtime.

# VI. Initial Model Training

## VI-0. Train Test Set Separation

`X` is the feature table and will all features except `actual_productivity`.

`y` is the label of the feature table and will only contain `actual_productivity`.

In [ ]:
X = df.drop(columns=['actual_productivity']).values
y = df['actual_productivity'].values

print('X ', X.shape)
print('y ', y.shape)

Divide the dataset into train and test sets, where `20%` of the data will be placed in the test set.

Random state is set to `69` for uniformity.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

Display the shape of the train, validation, and test sets.

In [ ]:
print('X_train', X_train.shape)
print('y_train', y_train.shape)

print('X_test', X_test.shape)
print('y_test', y_test.shape)

Display head and columns to check values

In [ ]:
print(pd.DataFrame(X_train).head())

As sklearn's StandardScaler standardizes all feature columns regardless of datatype, need to exclude boolean columns from the standardization to maintain their values

Get all non-boolean columns for standardization

In [ ]:
# Get all columns in X_train that are not boolean
non_boolean_columns = []
for i in range(X_train.shape[1]):
    if len(np.unique(X_train[:, i])) != 2:
        non_boolean_columns.append(i)

non_boolean_columns

Standardize the non-boolean columns based on X_train.

In [ ]:
# Scale all columns that are not boolean
ct = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), non_boolean_columns)
    ],
    remainder='passthrough'
)
ct.fit(X_train)

X_train_scaled = ct.transform(X_train)
X_test_scaled = ct.transform(X_test)

Display head and columns of standardizes datasets to check values

In [ ]:
print(pd.DataFrame(X_train_scaled).head())

Function to display MSE, RMSE, MAE, and R2 given a regressor

In [ ]:
def getModelPerformance(model, model_name:str, scaled:bool):
    
    # Predictions
    if scaled:
        y_train_pred = model.predict(X_train_scaled)
        y_test_pred = model.predict(X_test_scaled)
        
    else:
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
    
    # Clip the values to be between 0 and 1
    y_train_pred = np.clip(y_train_pred, 0, 1)
    y_test_pred = np.clip(y_test_pred, 0, 1)

    # MSE
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)

    print(model_name + ' Train MSE:', train_mse)
    print(model_name + ' Test MSE :', test_mse)
    print('')
    print(model_name + ' Train RMSE:', np.sqrt(train_mse))
    print(model_name + ' Test RMSE :', np.sqrt(test_mse))
    print('')
    print(model_name + ' Train MAE:', mean_absolute_error(y_train, y_train_pred))
    print(model_name + ' Test MAE :', mean_absolute_error(y_test, y_test_pred))
    print('')
    print(model_name + ' Train R^2:', r2_score(y_train, y_train_pred))
    print(model_name + ' Test R^2 :', r2_score(y_test, y_test_pred))

## VI-A. Linear Regression


Linear regression is a statistical and machine learning technique used to model the relationship between one or more independent variables (predictors) and a continuous dependent variable (target). It assumes a linear relationship between the predictors and the target, with the goal of finding the best-fitting line that minimizes the error between the predicted and actual values. Linear regression is simple to implement, computationally efficient, and interpretable, making it suitable for understanding relationships and making predictions. It performs well when the relationship between variables is approximately linear and is supported by statistical measures to evaluate the model's performance (Ross, 2010). However, it has limitations, such as sensitivity to outliers, reliance on assumptions like linearity and constant variance of errors, and difficulty handling multicollinearity or high-dimensional data. It may also struggle to capture complex or nonlinear relationships without additional transformations or feature engineering(Iqbal, 2020). 

In [ ]:
initial_linear_regression = LinearRegression()

Main Hyperparameters for Linear Regression

- fit_intercept: [True]
- normalize: [False] (deprecated in version 1.0, removed in 1.2)
- copy_X: [True]
- n_jobs: [None]

by setting normalize as true, Linear regression can benefit from feature scaling when the input variables have different scales (e.g., one feature is measured in meters and another in kilometers). It is important to note that Standard Scaling should be used instead of the old normalize hyperparameter in scikit linear regression.

Copy_X controls whether the model should make a copy of the feature matrix 𝑋 before fitting the model. By default, it ensures that the original data is not modified during the fitting process. If set to False, it can save memory if you're working with large datasets

In [ ]:
initial_linear_regression.fit(X_train_scaled, y_train)

In [ ]:
getModelPerformance(initial_linear_regression, 'Linear Regression', True)

## VI-B. Random Forest

Decision Trees have been used for classification and regression problems, however it faces the issue of being prone to bias and overfitting due to the nature of the algorithm. Therefore to remedy this we have decided to make use of the Random Forest model which is an ensemble learning method that makes use of decision trees.

The way random forests are able to improve its accuracy greatly over decision trees is due to various techniques with the first being bootstrapping to create a new dataset which can sample duplicates of the original dataset, and instead of considering all features of the dataset per split (i.e. all 24 columns for our dataset) it only considers a random subset of features (though the default values for sklearn consider all features), these two techniques in particular are what allow the individual decision trees to have little correlation with one another which helps with preventing overfitting.

Aggregating the decision of the multiple trees together leads to bagging, in the case of classification a majority vote is used to classify the unknown instance, whereas in regression the outputs of the trees will be averaged to get the value.

In [ ]:
initial_rf_regressor = RandomForestRegressor(random_state=random_seed)

Parameters:
- n_estimators: [100]
- max_depth: [None]
- min_samples_split: [2]
- min_samples_leaf: [1]
- min_weight_fraction_leaf: [0.0]
- max_features: [1.0]
- max_leaf_nodes: [None]
- min_impurity_decrease: [0.0]
- bootstrap: [True]
- oob_score: [False]
- n_jobs: [None]
- random_state: [69]
- verbose: [1]
- warm_start: [False]
- ccp_alpha: [0.0]
- max_samples: [None]
- monotonic_cst: [None]

The parameters above are all of the parameters that can be modified when creating the RandomForestRegressor(), however the main hyperparameters are:
- n_estimators
- max_depth
- max_features

Where n_estimators refers to the total number of trees that will be created by the model.

Where max_depth refers to the longest path that a model can create, which in its default state will create trees that go on until they are pure or until they reach min_samples_split.

Where max_features refers to the number of features to randomly sample at each split, which at default considers all features at each split (similar to a regular decision tree).

Some of these default values such as the n_estimators came from the paper that originally postulated the concept of the random forest (Breiman, 2001).

In [ ]:
initial_rf_regressor.fit(X_train, y_train)


In [ ]:
getModelPerformance(initial_rf_regressor, 'Initial Random Forest Model', False)

## VI-C. Neural Network

Neural Networks are a robust tool for predicting values due to their ability to capture complex, non-linear relationships between input features. 
With columns ranging from operational metrics like SMV, WIP, and over_time, to dynamic factors such as idle_time and no_of_style_change, Neural Networks can effectively process and learn from the intricate patterns and interactions among these variables. 
Additionally, their adaptability to various data scales and distributions makes them particularly suited to handle the diverse nature of the dataset, potentially leading to more accurate and insightful predictions of actual productivity. This capacity to model complex dependencies sets Neural Networks apart as a powerful predictive tool in this context.

To create the Neural Network model for the dataset, scikit-learn's Multi-layer Perceptron Regressor (MLP Regressor) will be used. 
It is a straightforward implementation of the supervised Neural Network model intended for simple use cases and learning, and it is complete with interconnected nodes, forwardpropagation, backpropagation, weight adjustment, and more.
Moreover, the MLP Regressor also comes with various hyperparameters that can be tuned to optimize performance and achieve high accuracy in predictive modeling tasks. (1.17. Neural Network Models (Supervised), n.d.).

SCRAPPED: Model would exploit clip and always go on top

Additionally, as the output of the regressor should only be between 0 and 1, to prevent it from having values beyond those, a custom MLPRegressor was made with with predictions being clipped at 0 and 1 before being returned.
By doing so, the model can train without needing to worry about going above or below the value ranges.

In [ ]:
# class ClippedMLPRegressor(BaseEstimator, RegressorMixin):
    
#     def __init__(self, **kwargs):
#         self.model = MLPRegressor(**kwargs)
#         self.max_iter = self.model.max_iter
    
#     def fit(self, X, y):
#         self.model.fit(X, y)
#         return self
    
#     def partial_fit(self, X, y):
#         self.model.partial_fit(X, y)
#         return self
    
#     def get_params(self, deep: bool = True) -> dict:
#         return self.model.get_params(deep)
    
#     def predict(self, X):
#         predictions = self.model.predict(X)
#         return np.clip(predictions, 0, 1)

In [ ]:
# Create MLPRegressor model
initial_nn_model = MLPRegressor(random_state=random_seed)

Main hyperparameters:
- Neurons and hidden layers: [100]
- Solver and Optimizer: ADAM
- Activation Function: ReLU
- Max Iterations: 200
- Initial Learning Rate: 0.001
- Learning Rate Change: constant
- Regularization Constant: 0.0001
- Random Sate: 69

The hyperparameters used for the initial model training are the default parameters of the MLPRegressor model. 
This is to set a base standard to which the model can improve upon when tuning. 
In general, it can be noted that the default parameters themselves are already sufficient for most use cases, already including advanced Neural Network optimizations like ADAM and ReLU.

Out of all the hyperparameters, the most significant is that there is only 1 hidden layer with 100 neurons. 
As the initial model, it is generally better to have a simple model with lesser neurons and hidden layers, then slowly increase the number over iterations.
This way, it is possible to see how the model improves with regards to the hyperparameters (Karpathy, 2019; Dutta, 2024).

In [ ]:
initial_nn_model.get_params()

To see how the model trains and improves, the training of the initial MLPRegressor will use partial_fit to fit it per epoch, and only stop when reach max iterations.
After each epoch, the error metrics for both train and test sets will be stored for evaluation later on, and at every 5th epoch, they will be displayed.
As the MLP Regressor model is sensitive to feature scaling, use the scaled versions of X_train and X_test to train the model. (1.17. Neural Network Models (Supervised), n.d.).

In [ ]:
# Use partial_fit to train the model and get the error metrics for each epoch

nn_epoch_num = 1
nn_epoch_train_mse = []
nn_epoch_train_mae = []
nn_epoch_r2 = []
nn_epoch_test_mse = []
nn_epoch_test_mae = []
nn_epoch_test_r2 = []

while True:
    
    # Train the model for one epoch
    initial_nn_model.partial_fit(X_train_scaled, y_train)
    
    # Predict the training set
    y_train_pred = initial_nn_model.predict(X_train_scaled)
    y_test_pred = initial_nn_model.predict(X_test_scaled)
    
    # Clip the values to be between 0 and 1
    y_train_pred = np.clip(y_train_pred, 0, 1)
    y_test_pred = np.clip(y_test_pred, 0, 1)
    
    # Calculate the error metrics
    train_rmse = mean_squared_error(y_train, y_train_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_rmse = mean_squared_error(y_test, y_test_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Append the mean squared error to the list
    nn_epoch_train_mse.append(train_rmse)
    nn_epoch_train_mae.append(train_mae)
    nn_epoch_r2.append(train_r2)
    nn_epoch_test_mse.append(test_rmse)
    nn_epoch_test_mae.append(test_mae)
    nn_epoch_test_r2.append(test_r2)
    
    # Print the epoch number and error metrics for every 10th epoch
    if nn_epoch_num % 10 == 0:
        print(f"Epoch: {nn_epoch_num}")
        print(f"\tTrain - RMSE: {np.sqrt(train_rmse)}, MAE: {train_mae}, R^2: {train_r2}")
        print(f"\tTest  - RMSE: {np.sqrt(test_rmse)}, MAE: {test_mae}, R^2: {test_r2}")
    
    # Break the loop if reach max_iter
    if nn_epoch_num == initial_nn_model.max_iter:
        break
    else:
        # Increment the epoch number
        nn_epoch_num += 1

After the model has been trained, display its error metrics for both train and test sets to show its performance. The analysis for the errors will be featured in the Error Analysis section.

In [ ]:
getModelPerformance(initial_nn_model, 'Initial Neural Network', True)

# VII. Error Analysis

## VII-0. Plotting Helper Functions

In [ ]:
def showPredToActualTable(y_test, y_pred, show_instances, is_test):
    string = 'Test' if is_test else 'Train'
    return pd.DataFrame({'Predicted Productivity': y_pred[:show_instances], string + ' Dataset Productivity': y_test[:show_instances]})

In [ ]:
def plotTrainingTestErrors(train_rmse, train_mae, test_rmse, test_mae):
    # Plot the training and testing errors
    plt.figure(figsize=(12, 6))
    plt.plot(train_rmse, label='Train RMSE', color='blue', linestyle="-")
    plt.plot(train_mae, label='Train MAE', color='blue', linestyle="--")
    plt.plot(test_rmse, label='Test RMSE', color='red', linestyle="-")
    plt.plot(test_mae, label='Test MAE', color='red', linestyle="--")
    plt.title('Training and Testing Errors')
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
def plotTrainingTestR2(train_r2, test_r2):
    # Plot the training and testing R^2
    plt.figure(figsize=(12, 6))
    plt.plot(train_r2, label='Train R^2', color='blue')
    plt.plot(test_r2, label='Test R^2', color='red')
    plt.title('Training and Testing R^2')
    plt.xlabel('Epoch')
    plt.ylabel('R^2')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
def plotProductivityError(y_test, y_pred, model_name, is_test):
    string = 'Test' if is_test else 'Train'
    indices = np.arange(len(y_pred))

    plt.figure(figsize=(25, 6))
    plt.plot(indices, y_pred, label=model_name+' Predicted Productivity', marker='', linestyle='-', color='blue', alpha=0.7)
    plt.plot(indices, y_test, label=string + ' Dataset Productivity', marker='', linestyle='--', color='red', alpha=0.7)

    plt.title("Comparison of Predicted vs " + string + " Dataset Productivity for " + model_name)
    plt.xlabel("Sample Index")
    plt.ylabel("Productivity")
    plt.legend()
    plt.grid(True)

    plt.show()

In [ ]:
def plotProductivityDistribution(y_test, y_pred, model_name, is_test):
    string = 'Test' if is_test else 'Train'
    min_value = min(np.min(y_test), np.min(y_pred))
    max_value = max(np.max(y_test), np.max(y_pred))
    bin_edges = np.linspace(min_value, max_value, 50)

    plt.figure(figsize=(12, 6))

    sns.histplot(y_pred, kde=True, color='blue', label=model_name+' Predicted Productivity', stat='percent', bins=bin_edges, alpha=0.6)
    sns.histplot(y_test, kde=True, color='red', label=string + ' Dataset Productivity', stat='percent', bins=bin_edges, alpha=0.6)

    plt.title("Distribution of Actual Productivity for " + model_name + " in " + string + " Dataset")
    plt.xlabel("Actual Productivity")
    plt.ylabel("Frequency (%)")
    plt.legend()

    plt.show()

## VII-A. Linear Regression

Based on the initial performance scores of the linear regression model, the model is relatively performing okay but with many instances of overfitting. The model was able to capture the various trends in the data but extreme deviations observed imply that the model struggles with noise in the dataset.

The presence of consistent variability across both graphs suggest that the model may further be improved using other techniques which may include regularization and further feature engineering.

In [ ]:
getModelPerformance(initial_linear_regression, 'Initial Linear Regression', True)

In [ ]:
lr_train_pred = initial_linear_regression.predict(X_train_scaled)
lr_test_pred = initial_linear_regression.predict(X_test_scaled)

print("Predicted - Train:\n")
print(showPredToActualTable(y_train, lr_train_pred, 20, False))
print("\n")
print("Predicted - Test:\n")
print(showPredToActualTable(y_test, lr_test_pred, 20, False))


In [ ]:
plotProductivityError(y_train, lr_train_pred, 'Initial Linear Regression (Train)', True)
plotProductivityError(y_test, lr_test_pred, 'Initial Linear Regression(Test)', True)


## VII-B. Random Forest

Seen below is the performance of the initial random forest regressor, where the performance of it is good for the training set, but it falls off greatly in the test set with its RMSE being much higher in the test set, and its R^2 score much lower where it matches only around 32% of the ground truth properly.

In [ ]:
getModelPerformance(initial_rf_regressor, 'Initial Random Forest', False)

In [ ]:
rf_train_pred = initial_rf_regressor.predict(X_train)
rf_test_pred = initial_rf_regressor.predict(X_test)

print("Predicted - Train:\n")
print(showPredToActualTable(y_train, rf_train_pred, 20, False))
print("\n")
print("Predicted - Test:\n")
print(showPredToActualTable(y_test, rf_test_pred, 20, False))

In [ ]:
plotProductivityError(y_train, rf_train_pred, 'Initial Random Forest (Train)', False)
plotProductivityError(y_test, rf_test_pred, 'Initial Random Forest(Test)', False)

plotProductivityDistribution(y_train, rf_train_pred, 'Initial Neural Network (Train)', False)
plotProductivityDistribution(y_test, rf_test_pred, 'Initial Neural Network (Test)', False)

## VII-C. Neural Network

In [ ]:
# Get the performance of the initial neural network model
getModelPerformance(initial_nn_model, 'Initial Neural Network', True)

Shown above are the final performance metrics of the Initial Neural Network after training.

For the train error metrics, it is not surprising that they are very low as the model is trained with it. Meanwhile for the test error metrics, given that it only used the default parameters for the MLP Regressor, its error metrics are already very low.

As the test RMSE and MAE are around 0.1~, it suggests that the the average error of the model's predicted value is +- 0.1~ from the test set's actual_productivity value.
However, the test R^2 error is 0.14, meaning that the model's predicted values only match around 14% of all the test set's actual_productivity values.
These values mean that the average error of the model to the test set is low, but it doesn't actually fit majority of the data trend well as a whole.

In [ ]:
# Get the predicted productivity for the train and test dataset
nn_y_train_pred = initial_nn_model.predict(X_train_scaled)
nn_y_test_pred = initial_nn_model.predict(X_test_scaled)

# Scale the predicted values back to 0 and 1
nn_y_train_pred = np.clip(nn_y_train_pred, 0, 1)
nn_y_test_pred = np.clip(nn_y_test_pred, 0, 1)

In [ ]:
# Show the first 20 instances of the predicted and actual productivity for the train and test dataset
print("Predicted - Train:\n")
print(showPredToActualTable(y_train, nn_y_train_pred, 20, False))
print("\n")
print("Predicted - Test:\n")
print(showPredToActualTable(y_test, nn_y_test_pred, 20, True))

Shown above are a subset of the predictions for the train and test sets of the Initial Neural Network.
This way we can observe the raw predicted values and how they compare to the actual ones.

In general, one can see that many of the predicted values for the train set are pretty close to the actual values, corresponding to the low train error metrics. 
However, going into the test set, there are usually large variances between the values, also corresponding to the test error metrics.

In [ ]:
plotTrainingTestErrors(np.sqrt(nn_epoch_train_mse), nn_epoch_train_mae, np.sqrt(nn_epoch_test_mse), nn_epoch_test_mae)
plotTrainingTestR2(nn_epoch_r2, nn_epoch_test_r2)

Shown above are the test and train error metrics of the model as it is trained per epoch.

For the train and test RMSE and MAE, they both followed the standard graph for decreasing their error values, suggesting they trained properly. 
But it can be noted that there is a small local-minima for the test set at around the 5th epoch that the model was able to overcome.
It can also be seen that the train error metrics are significantly lower than the test ones, highlighting how the model fits more to the train set due to being trained with it.
The same trend can also be seen with the train and test R^2 scores, with the train being higher than the test.

In [ ]:
plotProductivityError(y_train, nn_y_train_pred, 'Initial Neural Network (Train)', False)
plotProductivityError(y_test, nn_y_test_pred, 'Initial Neural Network (Test)', True)

Shown above are the comparisons of the predicted and actual productivity values of the Initial Neural Network to the train and test set.

Looking into the train set's graph, it can be seen how the model's predictions try its best to match those of the train set's actual productivity values. 
It was able to capture a certain trend of the data, thus matching it well in many of the points (although, it is quite hard to see).
However, it struggles to match the large variance of the higher values.

Using the model for the test set, one can see how the values don't match as well in the test set's graph.
Although it was able to follow a general trend, there were many cases where it failed to follow all the points.
Another observation was how the model's predictions tend to be more conservative as compared to the predicted values, opting to give less-varied productivity values.
Similar to the train set's graph, this makes it struggle to match the large variance of higher values; this may explain why the R^2 score was pretty low.

In [ ]:
plotProductivityDistribution(y_train, nn_y_train_pred, 'Initial Neural Network (Train)', False)
plotProductivityDistribution(y_test, nn_y_test_pred, 'Initial Neural Network (Test)', True)

Shown above are the histograms of the predicted and actual productivity values of the Initial Neural Network to the train and test set.
This visualizes the same data in another perspective to see more trends on the data.

Right out the bat, one can see how the data for the actual productivity is generally skewed to certain productivity scores, as shown by the distribution of the histogram.
Meanwhile, the predictions given by the model are a lot more conservative and spreads out its values.
These confirm the suspicion on the previous analysis. 

This unbalanced skew makes it difficult for the model to consistently predict the actual values of the dataset, resulting to worse error metrics, despite the trend lines in the graph being relatively similar.

# VIII. Improving Model Performance

## VIII-A. Linear Regression

In order to improve the performance of the linear regression, we used 2 ways of tuning the hyperparameters:

    1. Linear Regression with Lasso Regularization
    2. Stochastic Gradient Descent with Grid Search

Using both methods provides a broader range of optimization strategies, improves robustness, and ensures better adaptability to different data characteristics.

### Load Existing Models

Both models that are trained are stored in an external file, which is checked and loaded by the following cell. This is to avoid retraining the model, even if the models are trained relatively quickly.

In [ ]:
pipeline_lasso_gscv_model = None


sgd_lr_gscv_model = None

try:
    pipeline_lasso_gscv_model = joblib.load('pipeline_lasso_gscv.pkl')
except:
    pipeline_lasso_gscv_model = None

try: 
    sgd_lr_gscv_model = joblib.load('sgd_lr_gscv.pkl')
except:
    sgd_lr_gscv_model = None

    

### Linear Regression Pipeline using Lasso Regularization

Lasso Regularization was employed to reduce overfitting by penalizing large coefficients, resulting in a simpler model. By doing regularization, we hope to see if the model can generalize unseen and noise in the data better. Grid search is done for the alpha hyperparameter value to see what regularization value would yield the best performance. The closer the alpha value is to 0, the more similar this is to a simple linear regression without regularization (Tibshirani, 1996).

In [ ]:
pipeline_lasso_gscv_model = None

try:
    pipeline_lasso_gscv_model = joblib.load('pipeline_lasso_gscv.pkl')
except:
    pipeline_lasso_gscv_model = None

In [ ]:
lasso_gcsv_hyperparameters = None

if pipeline_lasso_gscv_model is None:
    lasso_gcsv_hyperparameters = {
        'alpha': [0.001, 0.01, 0.1, 1, 10, 100]
    }

In [ ]:
if pipeline_lasso_gscv_model is None:
    pipeline_lasso_gscv_model = Lasso()
    # implement grid search
    pipeline_lasso_gscv_model = GridSearchCV(pipeline_lasso_gscv_model, lasso_gcsv_hyperparameters, cv=5, verbose=1, n_jobs=-1)
    

In [ ]:
pipeline_lasso_gscv_model.fit(X_train_scaled, y_train)

In [ ]:
joblib.dump(pipeline_lasso_gscv_model, 'pipeline_lasso_gscv.pkl')

In [ ]:
pipeline_lasso_gscv_model.predict(X_train)

In [ ]:
pipeline_lasso_gscv_model.predict(X_test)

In [ ]:
getModelPerformance(pipeline_lasso_gscv_model, 'Pipeline Lasso Regression', True)

### Stochastic Gradient Descent with Grid Search

Stochastic Gradient Descent is an optimization algorithm used to minimize a cost function, which in our case is the mean squared error, by iteratively updating the model's parameters.

The randomness introduced by updating parameters using a single or small batch of samples reduces the risk for overfitting. By introducing randomness in the gradient updates, the noise prevents the model from converging too precisely to the training data's local minima (Jupudi, 2016).

In [ ]:
sgd_lr_gscv_hyperparameters = None

if sgd_lr_gscv_model is None:
    sgd_lr_gscv_hyperparameters = {
        'fit_intercept': [True, False],
        'max_iter': [200, 400, 600, 800, 1000],
        'learning_rate': ['adaptive', 'constant', 'invscaling'],
        'eta0': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
        'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1]
    }

Tune| the model using `GridSearchCV`

In [ ]:
if sgd_lr_gscv_model is None:
    sgd_lr_gscv_model = SGDRegressor(random_state=random_seed)
    sgd_lr_gscv_model = GridSearchCV(sgd_lr_gscv_model, sgd_lr_gscv_hyperparameters, cv=5, n_jobs=-1)

Train the model

In [ ]:
sgd_lr_gscv_model.fit(X_train_scaled, y_train)

Save the model to a file

In [ ]:
joblib.dump(sgd_lr_gscv_model, 'sgd_lr_gscv.pkl')

Get the best estimator of the model.

In [ ]:
sgd_lr_gscv_model.best_estimator_

Get the performance of the best estimator of the model.

In [ ]:
getModelPerformance(sgd_lr_gscv_model.best_estimator_, 'SGD with Grid Search', True)

## VIII-B. Random Forest

An issue that random forest models face is the more minimal effect of tuning compared to other models such as Support Vector Machines or Neural Networks, as something like n_estimators is dissimilar from other hyperparameters as it is almost always strictly better to go for higher values but only up to the point that tradeoff between performance and training time are not too great; while one of the hyperparameters that generally seem to have a greater effect is the number of random features considered at each split which in our case would be the max_features parameter(Probst et al., 2019). 

They've also suggested the usage of grid search, random search, and other tuning strategies to find optimal values for the hyper parameters, in our case we plan to implement random search.

In [ ]:
# Try to load the model
tuned_rf_model = None
try:
    tuned_rf_model = joblib.load('rf_rscv.pkl')
except:
    tuned_rf_model = None

In [ ]:
initial_rf_regressor = RandomForestRegressor(random_state=random_seed) #8, 12

Considering that the tradeoff for n_estimators is linear, its value will be continually increased until there has been a noticeable tradeoff on time, while for max depth the range of values has no particular study supporting the value range, however it should suffice considering the number of features in the dataset and the max depth exhibited by previous testing iterations. Another parameter that is important to tune is the max_features parameter as it controls the number of randomly selected features when splitting allowing the algorithm to make trees with less correlation leading to less overfitting.

The criterion for splitting used by the random forest may be changed from the default (mean squared error) to absolute error which uses MAE, and friedman_mse which uses the friedman_score which calculates the impurity of the split alongside the meansquared error. 

Although it is inherently part of the algorithm according to Breiman (2001), there are also some cases where the algorithm performs better when the bootstrapping does not occur, meaning that there are no duplicate instances in the subset created for the training of the random forest.

In [ ]:
rf_hyperparameters = [
        {
            'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
            'criterion': ['squared_error', 'absolute_error', 'friedman_mse'],
            'max_depth': [40, 50, 60, 70, 80, 90, 100],      
            'min_samples_leaf': [1, 3, 5],
            'max_features': [6, 12, 14, 18, 24,"sqrt", "log2"],
            'bootstrap': [True, False]
        }
    ]

In [ ]:
if tuned_rf_model is None:
    tuned_rf_model = RandomizedSearchCV(initial_rf_regressor, rf_hyperparameters, n_iter=500, cv=5, n_jobs=-1, random_state=random_seed)
    tuned_rf_model.fit(X_train, y_train)
    joblib.dump(tuned_rf_model, 'rf_rscv.pkl')

In [ ]:
pd.DataFrame(tuned_rf_model.cv_results_).sort_values('rank_test_score', ascending=True).head(10)

In [ ]:
tuned_rf_model.best_params_

From what can be inferred in the performance of the top 10 parameters values for the best parameters are as follows:
- n_estimators = [100, 200, 300, 500, 700, 800, 900]
- min_samples_leaf = [1]
- max_features = [6, 12, 14]
- max_depth = [50, 60, 70, 80, 90]
- criterion = ['squared error', 'absolute_error']
- bootstrap = [True, False]

Most of the high-performing parameters have n_estimators greater than the default value of 100 supporting the idea of convergence becoming better as the number of decision trees increase, whereas the min_samples_leaf remains similar to the default value where the final leaf nodes are pure. Another thing to note is that the value for maximum features tended towards lower values as a good chunk of the top 10 used max_features = 6, which allowed for randomizing features more. The splitting criterion was mainly split between mean squared error and mean absolute error amongst the top 10, where friedman score with MSE and poisson regression were not used as much, and the reason why mean absolute error was chosen over mean square error is likely due to the presence of the outliers as MAE is more robust against outliers. Finally, bootstrapping remained true for a majority of the models.

One thing to note is that the model has been able to generalize better and increased its performance on the test set with the hyper parameter tuning, even if the increase is only by 2% according to the R^2 score.

In [ ]:
getModelPerformance(tuned_rf_model, 'Initial Random Forest Model', False)

## VIII-C. Neural Network

After training the Initial Neural Network model and evaluating its performance, now it is time to create an improved model by using hyperparameter tuning.

As the hyperparameter tuning and training of Neural Networks takes a while to complete (40 minutes in this case), it was more economical to export and save the model for later use to prevent needing to retrain a new model every time the notebook is run.

The code below checks whether there already exists a Tuned Neural Network in the folder to use for testing and evaluation.
If there is already a saved model, it will not run the hyperparameter tuning anymore and will use the existing model.
Otherwise, it will train and tune a model, then save it into the folder.

In [ ]:
# Try to load the model
tuned_nn_model = None
try:
    tuned_nn_model = joblib.load('nn_rscv.pkl')
except:
    tuned_nn_model = None

Helper function to get all a list of all possible combinations of hidden layers given the minimum layers, maximum layers, and neuron options. 

In [ ]:
# Get all possible hidden layer sizes combinations
def get_hidden_layer_sizes(min_layers, max_layers, neuron_options):
    hidden_layer_sizes = []
    for num_layers in range(min_layers, max_layers + 1):
        for combination in itertools.product(neuron_options, repeat=num_layers):
            hidden_layer_sizes.append(combination)
    return hidden_layer_sizes

For the hyperparameter options, most of them were chosen with the basis being that they are the standard and most used.
This goes for the Initial Learning Rate and Regularization Constant being decimals of 1.
As for the max iterations, several standard values were included for the model to select the best number of iterations while preventing under and over fitting.

Meanwhile, as for the optimizer and activation function, they can be selected from those that are available within the MLPRegressor model.
In general, the most performant of the available options were selected, such as selecting ADAM over Stochastic Gradient Descent, TanH and ReLU over Logistic, and Adaptive over Constant learning rate change.

Lastly, the most significant of the hyperparameters is the number of hidden layers and neurons per layer. 
Although there is no definite answer for these parameters, a good starting point for simple problems is to have 1-2 hidden layers with 2/3 of the input size number for the neurons per layer (An, 2022).
However, the current problem is very intricate and complex with its values, so the simple case may not very feasible.
Additionally, as Python itself can output Memory Errors if a list is too big, certain values had to be limited for the model.
In the end, after multiple tunning iterations, it was found that 10 hidden layers with a combination of 50, 100, 250, and 500 neurons were the most performant in the hyperparameter tuning.
This was chosen to highlight the complexity of the features by adding multiple hidden layers, while also distributing neuron sizes to balance out the neurons for each layer.

Hyperparameter Tunning Options:
- Neurons and hidden layers: 10 layers with a combination of [50, 100, 250, 500] neurons
- Solver and Optimizer: ADAM
- Activation Function: [TanH, ReLU]
- Max Iterations: [200, 400, 600, 800, 1000]
- Initial Learning Rate: [0.1, 0.01, 0.001, 0.0001]
- Learning Rate Change: adaptive
- Regularization Constant: [0.0001, 0.001, 0.01, 0.1]
- Random Sate: 69

In [ ]:
nn_hyperparameters = None

# Define hyperparameters
if tuned_nn_model is None:
    
    # Define hidden layer sizes
    neuron_options = [50, 100, 250, 500]
    min_layers = 10
    max_layers = 10
    
    nn_hyperparameters = [
        {
            'hidden_layer_sizes': get_hidden_layer_sizes(min_layers, max_layers, neuron_options),
            'solver': ['adam'],
            'activation': ['tanh', 'relu'],
            'max_iter': [200, 400, 600, 800, 1000],
            'learning_rate_init': [0.1, 0.01, 0.001, 0.0001],
            'learning_rate': ['adaptive'],
            'alpha': [0.0001, 0.001, 0.01, 0.1]
        }
    ]

For the actual hyperparameter tuning technique, the RandomizedSearchCV technique was used.

RandomizedSearchCV is a method for tuning the hyperparameters of a model by randomly selecting values rather than using a fixed grid of parameters, like GridSearchCV. 
In each iteration, it tests a different set of hyperparameters and records how well the model performs by using Cross-Validation to separate the train set into train and validation and evaluate them. 
After multiple iterations, it identifies the combination that yields the best results. 
This random selection approach reduces unnecessary calculations compared to testing every possible combination systematically and can generally yield similar results (GeeksforGeeks, 2023).

To tune the hyperparameters faster, the n_jobs parameter was set to -1, which means that the model will use all available CPU cores to tune the model concurrently, thus saving time.
The n_iter parameter was also set to 100. 
Although this increases training time, it gives the RandomizedSearchCV more of a chance to stumble across a more accurate combination of hyperparameters.

In [ ]:
# Randomized Search to find the best hyperparameters and Train the model
if tuned_nn_model is None:
    
    tuned_nn_regressor = MLPRegressor(random_state=random_seed)
    tuned_nn_model = RandomizedSearchCV(tuned_nn_regressor, nn_hyperparameters, n_iter=100, cv=5, n_jobs=-1, random_state=random_seed)
    tuned_nn_model.fit(X_train_scaled, y_train)
    joblib.dump(tuned_nn_model, 'nn_rscv.pkl')

Display the top 10 best performing hyperparameter combinations.

In [ ]:
pd.DataFrame(tuned_nn_model.cv_results_).sort_values('rank_test_score', ascending=True).head(10)

Out of the top performing hyperparameter configurations, there is a clear gap between the mean test score performance of the top 5 combinations and the rest, with all of them being almost 40% accurate to the validation sets.
Of the top 5 combinations, there is a trend for them to have smaller learning rates and regularization constants.
This suggest that they might overfit to the training data due to training with them for many iterations and having little to combat high variance.
This may also explain why they might have achieved higher mean test scores.

Below are the best hyperparameters chosen after running the hyperparameter tunning.

Most Performant Hyperparameter Values:
- Neurons and hidden layers: 10 layers with [500, 250, 250, 100, 500, 500, 250, 500, 250, 50] neurons
- Solver and Optimizer: ADAM
- Activation Function: TanH
- Max Iterations: 1000
- Initial Learning Rate: 0.0001
- Learning Rate Change: adaptive
- Regularization Constant: 0.0001
- Random Sate: 69

In [ ]:
# Get the best hyperparameters
tuned_nn_model.best_params_

Based on the most performant hyperparameters, it can be implied that the Tuned Neural Network took a slower approach to its training; as highlighted by its very low initial learning rate and very high max iterations.
In most cases, one may think that there is a high chance that the model would overfit to the training set, as it was fitted to it through many iterations and that its regularization constant is very low.
Despite this, it still learned the trend of the data and was able to get pretty good predictions to the productivity values (to which will be discussed later).

After the hyperparameters have been tuned and the model has been trained, display its error metrics for both train and test sets to show its performance. The analysis for the errors will be featured in the Model Performance Summary section.

In [ ]:
# Get the performance of the model
getModelPerformance(tuned_nn_model, 'Tuned Neural Network', True)

# IX. Model Performance Summary

## IX-A. Linear Regression

Shown below are the performances of two models, i.e., Linear Regression with Lasso Regularization and Stochastic Gradient Descent. It is evidently observed that the two models perform with no significant improvement from an ordinary linear regression model. This may be further attributed to the high variability in a very small dataset. For both training and test dataset, there is observed overfitting, despite the model being able to capture the trends in the data.

In [ ]:
getModelPerformance(initial_linear_regression, 'Initial Linear Regression', True)

In [ ]:
getModelPerformance(pipeline_lasso_gscv_model.best_estimator_, 'Linear Regression with Lasso Regularization', True)

In [ ]:
pipeline_lasso_train_pred = pipeline_lasso_gscv_model.predict(X_train_scaled)
pipeline_lasso_test_pred = pipeline_lasso_gscv_model.predict(X_test_scaled)

print("Linear Regression with Lasso Regularization")
print("Predicted - Train:\n")
print(showPredToActualTable(y_train, pipeline_lasso_train_pred, 20, False))
print("\n")
print("Predicted - Test:\n")
print(showPredToActualTable(y_test, pipeline_lasso_test_pred, 20, True))
print("-----------------------------")

In [ ]:
plotProductivityError(y_train, pipeline_lasso_train_pred, 'Lasso Linear Regression (Train)', True)
plotProductivityError(y_test, pipeline_lasso_test_pred, 'Lasso Linear Regression (Test)', True)

plotProductivityDistribution(y_train, pipeline_lasso_train_pred, 'Lasso Linear Regression (Train)', True)
plotProductivityDistribution(y_test, pipeline_lasso_test_pred, 'Lasso Linear Regression (Test)', True)

It is shown that compared to the simple linear regression model, there is not much improvement in performance observed. For Lasso Regularization, the regularization parameter alpha was equal to 0.001, yielded the best performance. As the alpha value is too close to zero, the L1 penalty becomes negligible. In this scenario, Lasso will behave like a simple linear regression as there is no significant penalty to shrink coefficients.

In [ ]:
getModelPerformance(sgd_lr_gscv_model.best_estimator_, 'SGD with Grid Search', True)

In [ ]:
sgd_lr_gscv_train_pred = sgd_lr_gscv_model.predict(X_train_scaled)
sgd_lr_gscv_test_pred = sgd_lr_gscv_model.predict(X_test_scaled)

print("SGD - Grid Search")
print("Predicted - Train:\n")
print(showPredToActualTable(y_train, sgd_lr_gscv_train_pred, 20, False))
print("\n")
print("Predicted - Test:\n")
print(showPredToActualTable(y_test, sgd_lr_gscv_test_pred, 20, False))
print("-----------------------------")

In [ ]:
plotProductivityError(y_train, sgd_lr_gscv_train_pred, 'SGD with Grid Search (Train)', True)
plotProductivityError(y_test, sgd_lr_gscv_test_pred, 'SGD with Grid Search (Test)', True)

plotProductivityDistribution(y_train, sgd_lr_gscv_train_pred, 'SGD with Grid Search (Train)', True)
plotProductivityDistribution(y_test, sgd_lr_gscv_test_pred,  'SGD with Grid Search (Test)', True)

As we used SGD without regularization, it simply minimized the same cost function as a simple linear regression model, which is the mean squared error in this case. Hence, SGD will yield similar coefficeints and performance to a simple linear regression model. Moreover, with the dataset relatively small, the stochastic nature of SGD will not benefit the model, as this will behave like a full-batch gradient descent (equivalent to simple linear regression model).

The variability in the dataset is evidently observed. Because of this, linear regression struggles because it attempts to fit all the data including the noise leading to overfitting. Lasso and SGD adds bias into making the model simpler. This instead may cause the model to underfit. With this, linear regression, even with Lasso regularization and SGD, fails to perform well on small, high-variability datasets due to its inability handle noisy and nonlinear relationships. Regularization can mitigate overfitting to some degree but cannot compensate for the limited information in small datasets or reduce inherent variability in the target variable.

## IX-B. Random Forest

In [ ]:
getModelPerformance(tuned_rf_model, 'Initial Random Forest', False)

In [ ]:
rf_train_pred = tuned_rf_model.predict(X_train)
rf_test_pred = tuned_rf_model.predict(X_test)

print("Predicted - Train:\n")
print(showPredToActualTable(y_train, rf_train_pred, 20, False))
print("\n")
print("Predicted - Test:\n")
print(showPredToActualTable(y_test, rf_test_pred, 20, False))

In [ ]:
plotProductivityError(y_train, rf_train_pred, 'Initial Random Forest (Train)', False)
plotProductivityError(y_test, rf_test_pred, 'Initial Random Forest(Test)', False)

plotProductivityDistribution(y_train, rf_train_pred, 'Initial Neural Network (Train)', False)
plotProductivityDistribution(y_test, rf_test_pred, 'Initial Neural Network (Test)', False)

## IX-C. Neural Network

In [ ]:
# Get the performance of the initial and tuned neural network models
getModelPerformance(initial_nn_model, 'Initial Neural Network', True)
print('\n--------------------------------------------------\n')
getModelPerformance(tuned_nn_model, 'Tuned Neural Network', True)

Shown above are the final performance metrics of both the Initial and Tuned Neural Networks after training.

Comparing the performance of the Initial and Tuned Neural Networks, it can be noted that the Initial Model's predictions fits the train set a lot more than the Tuned Model. 
This may be because the RandomizedSearch using Cross-Validation, training the model with only a subset of the train set and using the rest for evaluation.
This lets the Tuned Model not overfit to the train set, thus why it has a higher error.

Meanwhile, for the test set, the Tuned Model performs only marginally better than the Initial Model.
This may be because the default parameters of the Initial Model are already pretty good in generating a workable model by themselves.
One thing to point out though is that the R^2 score of the Tuned Model is twice as good as the Initial Model's.
This suggests that the Tuned Model fits the test set's data trend more as a whole.

In [ ]:
nn_y_train_pred = tuned_nn_model.predict(X_train_scaled)
nn_y_test_pred = tuned_nn_model.predict(X_test_scaled)

# Scale the predicted values back to 0 and 1
nn_y_train_pred = np.clip(nn_y_train_pred, 0, 1)
nn_y_test_pred = np.clip(nn_y_test_pred, 0, 1)

In [ ]:
plotProductivityError(y_train, nn_y_train_pred, 'Tuned Neural Network (Train)', False)
plotProductivityError(y_test, nn_y_test_pred, 'Tuned Neural Network (Test)', True)

Shown above are the comparisons of the predicted and actual productivity values of the Tuned Neural Network to the train and test set.

In general, the Tuned Model's predictions show many of the same qualities of those of the Initial Model.
Mainly that is still conservative with its predicted productivity values, in contrast to the large variance of the actual productivity values in the dataset.
However, a somewhat key difference is that the model performed worse in the test set compared to the Initial Model, as mentioned in the error metrics.
In exchange, the Tuned Model now better follows the general trend of the data, with its predictions being able to follow the actual values in most cases, in exception to the large variance data points. 

In [ ]:
plotProductivityDistribution(y_train, nn_y_train_pred, 'Tuned Neural Network (Train)', False)
plotProductivityDistribution(y_test, nn_y_test_pred, 'Tuned Neural Network (Test)', True)

Shown above are the histograms of the predicted and actual productivity values of the Tuned Neural Network to the train and test set.

Similar to the analysis above, the Tuned Model's predictions show many of the same qualities of those of the Initial Model.
The key difference of the Tuned Model to the Initial Model's historgram is that the Tuned Model's predictions are more spread out in comparison.
This highlights how it prefers to make conservative predictions based on certain trends in the data and distribute its values.
However, this approach may not be the most effective as the distribution of the actual productivity of the dataset tends to skew to certain values, as shown in the histogram.

# X. Insights and Conclusions

# XI. References

## Neural Networks
1. 1.17. Neural network models (supervised). (n.d.). Scikit-learn. https://scikit-learn.org/stable/modules/neural_networks_supervised.html#neural-networks-supervised
   - Multi-layer Perceptron (MLP) Models of scikit-learn

2. MLPRegressor. (n.d.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor
   - Multi-layer Perceptron (MLP) Regressor of scikit-learn

3. Karpathy, A. (2019, April 25). A recipe for training neural networks. Andrej Karpathy blog. https://karpathy.github.io/2019/04/25/recipe/
   - General tips to train Neural Networks

4. Dutta, S. (2024, November 24). Number of neurons per hidden layer in neural networks: a guide. Medium. https://medium.com/@sanjay_dutta/number-of-neurons-per-hidden-layer-in-neural-networks-a-guide-106fea04fbfe
   - Tips for number of neurons and hidden layers for Neural Networks

5. An, S. (2022, June 2). How to tune hyperparameters for better neural network performance. Medium. https://medium.com/codex/how-to-tune-hyperparameters-for-better-neural-network-performance-b8f542855d2e
   - Tips for hyperparameter tuning of Neural Networks

6. GeeksforGeeks. (2023, December 7). Hyperparameter tuning. GeeksforGeeks. https://www.geeksforgeeks.org/hyperparameter-tuning/
   - Tips for hyperparameter tuning in general

7. Functions of Finishing Department in Garment Industry. (2015, October 24). Online Clothing Study. https://www.onlineclothingstudy.com/2015/10/functions-of-finishing-department-in.html
   - Some info on what the finishing department does

8. Breiman, L. (2001). Random Forests. Machine Learning, 45(1), 5–32. https://doi.org/10.1023/A:1010933404324
   - Original Random Forest paper, also explains the convergence and prevention of overfitting

9. Probst, P., Wright, M. N., & Boulesteix, A.-L. (2019). Hyperparameters and tuning strategies for random forest. WIREs Data Mining and Knowledge Discovery, 9(3), e1301. https://doi.org/10.1002/widm.1301
   - RF Tuning strategies 1

10. Scornet, E. (2017). Tuning parameters in random forests. ESAIM: Proceedings and Surveys, 60, 144–162. https://doi.org/10.1051/proc/201760144
   - RF Tuning strategies 2

11. What Is Random Forest? | IBM. (2021, October 20). https://www.ibm.com/topics/random-forest
   - Surface Level Understanding of RF

12. Ross, Sheldon M. “Linear Regression.” Introductory Statistics, 2010, pp. 537–604, www.sciencedirect.com/science/article/pii/B9780123743886000120, https://doi.org/10.1016/b978-0-12-374388-6.00012-0.
- Surface Level Understanding of Linear Regression

13. Iqbal, M. (2020). Application of Regression Techniques with their Advantages and Disadvantages. https://www.researchgate.net/profile/Muhammad-Ahmad-Iqbal/publication/354921553_Application_of_Regression_Techniques_with_their_Advantages_and_Disadvantages/links/61543c3c14d6fd7c0fb91053/Application-of-Regression-Techniques-with-their-Advantages-and-Disadvantages.pdf
- Advantages and Disadvantages of Linear Regression

14. Tibshirani, R. (1996). Regression Shrinkage and Selection Via the Lasso. Journal of the Royal Statistical Society: Series B (Methodological), 58(1), 267–288. https://doi.org/10.1111/j.2517-6161.1996.tb02080.x
- Lasso Regularization for Linear Regression

15. Jupudi, Lakshmi(2016). Stochastic Gradient Descent using Linear Regression with Python. IJAERA. 2. 519 -525. 
- Stochastic Gradient Descent for Linear Regression